In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [20]:
# Wczytanie danych
df = pd.read_csv("dataset_ML_v1.csv")  # Podmień nazwę pliku na właściwą

In [21]:
# Usunięcie ewentualnych brakujących wartości
df.dropna(inplace=True)

In [22]:
# Wybór cech (featureów) i etykiet (label)
X = df[['Zamkniecie', 'STS', 'MACD', 'MACD_Signal', 'RSI']]
y = df['Label']

In [23]:
# Podział na zestaw treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Inicjalizacja i trenowanie modelu RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [25]:
# Ocena modelu
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność modelu: {accuracy:.2f}")

Dokładność modelu: 0.76


In [26]:
y_proba = model.predict_proba(X_test)
df_results = pd.DataFrame(y_proba, columns=["Spadek", "Wzrost"])
df_results["Rekomendacja"] = df_results["Wzrost"].apply(lambda x: "Kup" if x > 0.6 else ("Sprzedaj" if x < 0.4 else "Brak decyzji"))
print(df_results)


      Spadek  Wzrost  Rekomendacja
0       0.00    1.00           Kup
1       1.00    0.00      Sprzedaj
2       0.99    0.01      Sprzedaj
3       0.72    0.28      Sprzedaj
4       0.01    0.99           Kup
...      ...     ...           ...
1742    0.34    0.66           Kup
1743    0.22    0.78           Kup
1744    0.87    0.13      Sprzedaj
1745    0.40    0.60  Brak decyzji
1746    0.00    1.00           Kup

[1747 rows x 3 columns]


In [27]:
# dodawanie rekomendacji do wsszystkich danych
y_proba = model.predict_proba(X)  # Użycie całego datasetu zamiast X_test
df["Prawdopodobieństwo_Spadku"] = y_proba[:, 0]
df["Prawdopodobieństwo_Wzrostu"] = y_proba[:, 1]
df["Rekomendacja"] = df["Prawdopodobieństwo_Wzrostu"].apply(lambda x: "Kup" if x > 0.6 else ("Sprzedaj" if x < 0.4 else "Brak decyzji"))

In [37]:
print(df)

           Data  Zamkniecie     Wolumen        Date  \
0      1/3/2023     3824.14  2351704257    1/3/2023   
1      1/4/2023     3852.97  2574150675    1/4/2023   
2      1/5/2023     3808.10  2254045861    1/5/2023   
3      1/6/2023     3895.08  2462500481    1/6/2023   
4      1/9/2023     3892.09  2498158582    1/9/2023   
..          ...         ...         ...         ...   
494  12/24/2024     6040.04  1133216588  12/24/2024   
495  12/26/2024     6037.59  1506250902  12/26/2024   
496  12/27/2024     5970.84  1817797977  12/27/2024   
497  12/30/2024     5906.94  1943393216  12/30/2024   
498  12/31/2024     5881.63  1796606528  12/31/2024   

     Daily index price Change (%)         STS       MACD  MACD_Signal  \
0                       -0.400052   17.307448 -29.174369   -19.077689   
1                        0.753895   32.885431 -27.032923   -20.668736   
2                       -1.164556   22.109660 -28.626456   -22.260280   
3                        2.284079  100.000000 -

In [38]:
# Saving dataset with recommendation
df.to_csv('dataset_recommendation.csv', index=False)

In [39]:
# Badanie skutecznosci systemu

In [46]:
# Wczytanie danych
df = pd.read_csv("dataset_recommendation_2023.csv")

# Założenia początkowe
initial_budget = 5000  # Początkowy kapitał inwestycyjny w PLN

# Obliczenie zwrotu procentowego
df['Return'] = df['Zamkniecie'].pct_change()

# Filtrowanie transakcji na podstawie rekomendacji
df['Signal'] = df['Rekomendacja'].map({'Kup': 1, 'Sprzedaj': -1})

# Obliczenie efektywności strategii
df['Strategy Return'] = df['Signal'] * df['Return']

# Obliczenie wartości portfela w czasie
df['Portfolio Value'] = initial_budget * (1 + df['Strategy Return']).cumprod()

# Wyświetlenie wyników
print(f"Wartość portfela na koniec analizy: {df['Portfolio Value'].iloc[-1]:.2f} PLN")


Wartość portfela na koniec analizy: 85985.47 PLN
